In [ ]:
import numpy as np
import random
import sys
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import cv2
from torchsummary import summary

In [ ]:
import gym
env = gym.make("Breakout-ramDeterministic-v4")


print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Box(128,)
Action space: Discrete(4)


In [ ]:
cap=200000
batch_size=32
lr=1e-5
eps=1
n_ep=20000
gamma=0.99
no_of_actions=4
state_size=128


In [ ]:
class ReplayMemory:
    def __init__(self, max_size):
        self.buffer = [None] * max_size
        self.max_size = max_size
        self.index = 0
        self.size = 0

    def append(self, obj):
        self.buffer[self.index] = obj
        self.size = min(self.size + 1, self.max_size)
        self.index = (self.index + 1) % self.max_size

    def sample(self, batch_size):
        indices = random.sample(range(self.size), batch_size)
        return [self.buffer[index] for index in indices]

In [ ]:
 def init_weights(m):
        if type(m) == nn.Linear:
            torch.nn.init.uniform_(m.weight, -0.01, 0.01)
            m.bias.data.fill_(0.01)
            

In [ ]:
def plotter(s,p):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot(np.arange(len(s)), s)
        plt.ylabel('av_score')
        plt.xlabel('Episode*50')
        plt.show()

        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot(np.arange(len(p)), p)
        plt.ylabel('q_values')
        plt.xlabel('episode*20')
        plt.show()


In [ ]:
def save_model():
        model_save_name = 'breakout1.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}" 
        checkpoint = {
                        'model_state_dict':agent.model.state_dict(),
                        'target_state_dict':agent.target.state_dict(),
                        'optimizer_state_dict': agent.optimizer.state_dict(),
                        'memory':agent.memory
                        }

        torch.save(checkpoint, path)
def save_model2():
        model_save_name = 'breakout2.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}" 
        checkpoint = {
                        'model_state_dict':agent.model.state_dict(),
                        'target_state_dict':agent.target.state_dict(),
                        'optimizer_state_dict': agent.optimizer.state_dict(),
                        'memory':agent.memory
                        }

        torch.save(checkpoint, path)
def save_agent():
        model_save_name = 'breakout_agent.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}" 
        checkpoint = {
                        'n':n,
                        'i':i,
                        'eps': eps,
                        's':s,
                        'p':p,
                        'av':scores
                        }

        torch.save(checkpoint, path)
def load_agent():
      model_save_name = 'breakout_agent.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}" 
      checkpoint=torch.load(path)
      n=checkpoint['n']
      print(n)
      i=checkpoint['i']
      s=checkpoint['s']
      p=checkpoint['p']
      scores=checkpoint['av']
      eps=checkpoint['eps']

def load_tests():
    model_save_name = 'breakout_test.pt'
    per=torch.load(F"/content/gdrive/My Drive/{model_save_name}" )
    per=(per['test_states'])
    test_states= np.zeros((20000,512))
    test_states=np.concatenate(per, out=test_states)
    test_states=torch.from_numpy(test_states).float()
    return test_states
def save_tests():
      model_save_name = 'breakout_test.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}" 
      checkpoint = {
                      'test_states':test_states
                      }

      torch.save(checkpoint, path)
def load_model():
      model_save_name = 'breakout1.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}" 
      checkpoint=torch.load(path)
      agent.model.load_state_dict(checkpoint['model_state_dict'])
      agent.target.load_state_dict(checkpoint['target_state_dict'])
      agent.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      agent.memory=(checkpoint['memory'])


In [ ]:
class dqn(nn.Module):
    def __init__(self,state_size,no_of_actions):
        
        super().__init__()
        self.fc1 = nn.Linear(128*4, 256)
        self.fc2=nn.Linear(256,128)
        self.fc3=nn.Linear(128,64)
        self.fc4=nn.Linear(128,64)
        self.fc5 = nn.Linear(64, no_of_actions)
        self.fc6 = nn.Linear(64 , 1)

    def forward(self, x):
        out = self.fc1(x)
        out=F.relu(out)
        out = self.fc2(out)
        out=F.relu(out)
        V = F.relu(self.fc3(out))
        v =self.fc6(V)
        A=F.relu(self.fc4(out))
        a = self.fc5(A)
        q=v+(a-torch.mean(a, dim=1, keepdim=True))
        return q
    
   


In [ ]:
class agent:
    def __init__(self):
    
        self.memory=ReplayMemory(cap)
        self.model=dqn(state_size,no_of_actions)
        self.target=dqn(state_size,no_of_actions)
        self.target.apply(init_weights)
        self.model.apply(init_weights)
        self.optimizer=optim.Adam(self.model.parameters(),lr)
        self.x=0
      
    
    def remember(self,state,action,reward,next_state,done):
      
        self.memory.append([state,action,reward,next_state,done])
        
    def sample(self):
        
        experiences=self.memory.sample(batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float()
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long()
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float()
        next_states =  torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float()
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float()
        return (states, actions, rewards, next_states, dones)
    
   
    def train(self):
        states, actions, rewards, next_states, dones = self.sample()
        Q=self.model(states).gather(1, actions)
        _,a=torch.max(self.model(next_states),1,keepdim=True)
        Q_next_state=self.target(next_states).gather(1, a)
        Q_target=rewards+gamma*(Q_next_state*(1-dones))
        loss=F.mse_loss(Q,Q_target)
        self.x+=1
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        if(self.x%10000==0):
            self.update(self.model,self.target)

   
    def action(self,state,eps):
       
        if random.random() > eps:
            with torch.no_grad():
                state= torch.from_numpy(state).float()
                action_values = self.model(state)
            action= (np.argmax(action_values).item())
        else:
            action= random.choice(np.arange(no_of_actions))
        return action
    
    def update(self,local,target):
         for target_param, local_param in zip(target.parameters(), local.parameters()):
            target_param.data.copy_(local_param.data )
    

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
agent=agent()
summary(agent.model,(1,512))
test_states=load_tests()
scores=deque(maxlen=100)
i=0
s=[]
p=[]

for n in range(0,n_ep):
    h=0
    
    score=0
    state=np.zeros((1,3*128))
    obs=env.reset()
    obs=obs.reshape(1,128)
    state=np.concatenate((state,obs/255),axis=1)
    done=False
    li=5
    while True:
        if(i<50000):
          eps=1
        elif(i<1050000):
          eps-=0.9/1000000
        elif(i<2050000):
          eps-=0.05/1000000
        action= agent.action( state, eps)
        next_obs,reward,done,_=env.step(action)
        lf=env.ale.lives()
        score+=reward
        next_obs=next_obs.reshape(1,128)
        if(li==lf):
            next_state=state[:,128:].copy()
            next_state=np.concatenate([next_state,next_obs/255.0], axis = 1)
            agent.remember(state,action,reward,next_state,done)
        else:
            li=lf
            next_state=np.zeros((1,3*128))
            next_state=np.concatenate([next_state,next_obs/255.0], axis = 1)
            agent.remember(state,action,reward,next_state,1)
        
        i+=1

        state=next_state.copy()
        h+=1
        if((i+1)>batch_size):
            agent.train()
        if done:
            break

    if((n+1)%20==0):
        with torch.no_grad():
            v=(torch.max(agent.model(test_states),1,keepdim=True)).values
        p.append(v.mean())

    scores.append(score)
    avg_reward = np.mean(scores)
    print('Game {}, Score = {},Steps = {}, av_score={}'.format(n+1,score,h,avg_reward))
    if((n+1)%50==0):
        s.append(avg_reward)
    if((n+1)%300==0): 
        print(i,eps)
        save_agent()
        if((n+1)%600==0):
            save_model()
        else:
            save_model2()
        plotter(s,p)   

In [ ]:
def record(out):
    image = env.unwrapped._get_image()
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    out.write(image)
    return out
    
    
k=[]
eps=0.05

for n in range(0,20):
    h=0
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/content/gdrive/My Drive/breakout' + str(_) + '.avi',fourcc,20, (160, 210))
    score=0
    state=np.zeros((1,3*128))
    obs=env.reset()
    out = record(out)
    obs=obs.reshape(1,128)
    state=np.concatenate((state,obs/255),axis=1)
    done=False
    li=5
    while True:
        action= agent.action( state, eps)
        next_obs,reward,done,_=env.step(action)
        out = record(out)
        lf=env.ale.lives()
        score+=reward
        next_obs=next_obs.reshape(1,128)
        if(li==lf):
            next_state=state[:,128:].copy()
            next_state=np.concatenate([next_state,next_obs/255.0], axis = 1)
            
        else:
            li=lf
            next_state=np.zeros((1,3*128))
            next_state=np.concatenate([next_state,next_obs/255.0], axis = 1)
            
        
       

        state=next_state.copy()
        h+=1
        
        if done:
            break

   

    k.append(score)
    avg_reward = np.mean(k)
    print('Game {}, Score = {},Steps = {}, av_score={}'.format(n+1,score,h,avg_reward))